In [1]:
import numpy as np
import pandas as pd
import plotly as plt
import scipy
import math
from datetime import datetime

In [2]:
df_matches = pd.read_csv("../data/cleaned/tennis_matches_cleaned.csv", parse_dates=['tourney_date'])
df_male_players = pd.read_csv("../data/cleaned/male_players_cleaned.csv")
df_female_players = pd.read_csv("../data/cleaned/female_players_cleaned.csv")

# Data integration

Create a dataframe with all the male matches and another one for the female matches

In [3]:
df_male_players_tmp = pd.DataFrame()
df_male_players_tmp['player_name'] = df_male_players['name'] + ' ' + df_male_players['surname']
df_male_winners = df_male_players_tmp.join(df_matches.set_index('winner_name'), on='player_name', how='inner')
df_male_losers = df_male_players_tmp.join(df_matches.set_index('loser_name'), on='player_name', how='inner')
df_male_matches = pd.concat([df_male_winners, df_male_losers])

In [4]:
df_male_matches

,player_name,Unnamed: 0,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,...,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,tourney_spectators,tourney_revenue,winner_name
643,Alexander Zverev,352,2019-580,Australian Open,Hard,128,G,2019-01-14,200,100644,...,13.0,6.0,12.0,4,6385.0,155,343.0,15583.0,2781000.18,NaN
643,Alexander Zverev,363,2019-580,Australian Open,Hard,128,G,2019-01-14,116,100644,...,13.0,2.0,8.0,4,6385.0,67,760.0,15583.0,2781000.18,NaN
643,Alexander Zverev,364,2019-580,Australian Open,Hard,128,G,2019-01-14,172,100644,...,26.0,12.0,17.0,4,6385.0,36,1120.0,15583.0,2781000.18,NaN
643,Alexander Zverev,1145,2019-M004,Acapulco,Hard,32,A,2019-02-25,298,100644,...,10.0,2.0,4.0,3,6475.0,64,797.0,3398.0,655207.08,NaN
643,Alexander Zverev,1149,2019-M004,Acapulco,Hard,32,A,2019-02-25,294,100644,...,10.0,0.0,2.0,3,6475.0,26,1443.0,3398.0,655207.08,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54476,Michele Vianello,94508,2021-9357,Forli CH,Clay,32,C,2021-06-14,243,104563,...,9.0,2.0,6.0,433,93.0,-1,NaN,3031.0,566031.89,Luca Vanni
54476,Michele Vianello,95338,2021-2120,Cordenons CH,Clay,32,C,2021-08-02,250,207608,...,10.0,2.0,7.0,291,195.0,-1,NaN,2876.0,543907.53,Timofey Skatov
54485,Ben Shelton,95003,2021-7316,Cary CH,Hard,32,C,2021-07-19,254,206499,...,9.0,3.0,5.0,413,100.0,1835,1.0,3074.0,583894.56,Aleksandar Kovacevic
54504,William Jansen,64393,2021-540,Wimbledon,Grass,128,G,2021-06-28,138,207760,...,13.0,8.0,16.0,454,87.0,-1,NaN,14566.0,5002793.97,Anton Matusevich


df_male_matches.describe()

In [5]:
df_female_players_tmp = pd.DataFrame()
df_female_players_tmp['player_name'] = df_female_players['name'] + ' ' + df_female_players['surname']
df_female_winners = df_female_players_tmp.join(df_matches.set_index('winner_name'), on='player_name', how='inner')
df_female_losers = df_female_players_tmp.join(df_matches.set_index('loser_name'), on='player_name', how='inner')
df_female_matches = pd.concat([df_female_winners, df_female_losers])

In [6]:
df_male_matches_reduced = df_male_losers[['player_name', 'winner_id', 'winner_ht', 'winner_age', 'tourney_date']]
df_male_matches_reduced[df_male_matches_reduced['player_name'] == 'Aaron Addison']

,player_name,winner_id,winner_ht,winner_age,tourney_date
31893,Aaron Addison,117357,-1,19,2016-10-24
31893,Aaron Addison,117360,-1,18,2016-02-01
31893,Aaron Addison,102864,193,-1,2016-01-11
31893,Aaron Addison,105429,-1,26,2016-10-31
31893,Aaron Addison,111806,-1,23,2020-01-13
31893,Aaron Addison,105349,-1,30,2019-10-21
31893,Aaron Addison,106172,-1,25,2019-01-07
31893,Aaron Addison,200614,-1,21,2019-10-28
31893,Aaron Addison,200640,-1,19,2018-10-22


- name
- hand
- sex
- nationality

- win percentage
- tot match
- number of won tournment

- year with more matches
- year with less matches
- year with more wins
- year with less wins

Reset the index of both the new data frames

In [7]:
df_male_matches.reset_index(drop=True, inplace=True)
df_female_matches.reset_index(drop=True, inplace=True)

In [8]:
df_player = pd.DataFrame()

list_of_male_names = df_male_matches['player_name'].unique()
print(f"Lunghezza lista: {len(list_of_male_names)}")
df_player['Name'] = list_of_male_names
df_player['Sex'] = 'M'
i = 0

for name in list_of_male_names:
    print(i)
    i+=1
    df_player_matches = df_matches[(df_matches['winner_name'] == name) | (df_matches['loser_name'] == name)]

    #player's hand
    df_h = [x for x in df_player_matches[(df_player_matches['winner_name'] == name) &  (df_player_matches['winner_hand'] != 'U')]['winner_hand']]
    if len(df_h) > 0:
        df_player.loc[df_player['Name'] == name, 'Hand'] = df_h[0]
    else:
        df_h = [x for x in df_player_matches[(df_player_matches['loser_name'] == name) &  (df_player_matches['loser_hand'] != 'U')]['loser_hand']]
        if len(df_h) > 0:
            df_player.loc[df_player['Name'] == name, 'Hand'] = df_h[0]
        else:
            df_player.loc[df_player['Name'] == name, 'Hand'] = 'U'

    #player's nationality
    df_h = [x for x in df_player_matches[(df_player_matches['winner_name'] == name) &  (df_player_matches['winner_ioc'] != None)]['winner_ioc']]
    if len(df_h) > 0:
        df_player.loc[df_player['Name'] == name, 'IOC'] = df_h[0]
    else:
        df_h = [x for x in df_player_matches[(df_player_matches['loser_name'] == name) &  (df_player_matches['loser_ioc'] != 'U')]['loser_ioc']]
        if len(df_h) > 0:
            df_player.loc[df_player['Name'] == name, 'IOC'] = df_h[0]
        else:
            df_player.loc[df_player['Name'] == name, 'IOC'] = 'Undefined'
    
    df_player.loc[df_player['Name'] == name, 'Won_Tournaments'] = df_player_matches[(df_player_matches['round'] == 'F') & (df_player_matches['winner_name'] == name)].shape[0]

    if df_player_matches.shape[0] != 0:
        df_player.loc[df_player['Name'] == name, 'Number_of_Matches'] = df_player_matches.shape[0]
        df_player.loc[df_player['Name'] == name, 'Win_Percentage'] = df_player_matches[df_player_matches['winner_name'] == name].shape[0]/df_player_matches.shape[0]
    
    match_per_year = df_male_matches[df_male_matches['player_name']==name]['tourney_date'].groupby(df_male_matches['tourney_date'].dt.year).count()
    if len(match_per_year) > 0:
        df_player.loc[df_player['Name'] == name, 'best_year_matches'] = match_per_year.idxmax()
        df_player.loc[df_player['Name'] == name, 'worst_year_matches'] = match_per_year.idxmin()

    wins_per_year = df_male_matches[df_male_matches['winner_name']==name]['tourney_date'].groupby(df_male_matches['tourney_date'].dt.year).count()
    if len(wins_per_year) > 0:
        df_player.loc[df_player['Name'] == name, 'best_year_wins'] = wins_per_year.idxmax()
        df_player.loc[df_player['Name'] == name, 'worst_year_wins'] = wins_per_year.idxmin()




Lunghezza lista: 3018
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271


In [9]:
df_player.to_csv("../data/integrations/matches_male_players.csv", index=False)

In [10]:
df_player = pd.DataFrame()

list_of_female_names = df_female_matches['player_name'].unique()
print(list_of_female_names)
df_player['Name'] = list_of_female_names
df_player['Sex'] = 'F'
i = 0

for name in list_of_female_names:
    
    df_player_matches = df_matches[(df_matches['winner_name'] == name) | (df_matches['loser_name'] == name)]

    #player's hand
    df_h = [x for x in df_player_matches[(df_player_matches['winner_name'] == name) &  (df_player_matches['winner_hand'] != 'U')]['winner_hand']]
    if len(df_h) > 0:
        df_player.loc[df_player['Name'] == name, 'Hand'] = df_h[0]
    else:
        df_h = [x for x in df_player_matches[(df_player_matches['loser_name'] == name) &  (df_player_matches['loser_hand'] != 'U')]['loser_hand']]
        if len(df_h) > 0:
            df_player.loc[df_player['Name'] == name, 'Hand'] = df_h[0]
        else:
            df_player.loc[df_player['Name'] == name, 'Hand'] = 'U'

    #player's nationality
    df_h = [x for x in df_player_matches[(df_player_matches['winner_name'] == name) &  (df_player_matches['winner_ioc'] != None)]['winner_ioc']]
    if len(df_h) > 0:
        df_player.loc[df_player['Name'] == name, 'IOC'] = df_h[0]
    else:
        df_h = [x for x in df_player_matches[(df_player_matches['loser_name'] == name) &  (df_player_matches['loser_ioc'] != 'U')]['loser_ioc']]
        if len(df_h) > 0:
            df_player.loc[df_player['Name'] == name, 'IOC'] = df_h[0]
        else:
            df_player.loc[df_player['Name'] == name, 'IOC'] = 'Undefined'
    
    df_player.loc[df_player['Name'] == name, 'Won_Tournaments'] = df_player_matches[(df_player_matches['round'] == 'F') & (df_player_matches['winner_name'] == name)].shape[0]

    if df_player_matches.shape[0] != 0:
        df_player.loc[df_player['Name'] == name, 'Number_of_Matches'] = df_player_matches.shape[0]
        df_player.loc[df_player['Name'] == name, 'Win_Percentage'] = df_player_matches[df_player_matches['winner_name'] == name].shape[0]/df_player_matches.shape[0]

    match_per_year = df_female_matches[df_female_matches['player_name']==name]['tourney_date'].groupby(df_female_matches['tourney_date'].dt.year).count()
    if len(match_per_year) > 0:
        df_player.loc[df_player['Name'] == name, 'best_year_matches'] = match_per_year.idxmax()
        df_player.loc[df_player['Name'] == name, 'worst_year_matches'] = match_per_year.idxmin()

    wins_per_year = df_female_matches[df_female_matches['winner_name']==name]['tourney_date'].groupby(df_female_matches['tourney_date'].dt.year).count()
    if len(wins_per_year) > 0:
        df_player.loc[df_player['Name'] == name, 'best_year_wins'] = wins_per_year.idxmax()
        df_player.loc[df_player['Name'] == name, 'worst_year_wins'] = wins_per_year.idxmin()

['Mirjana Lucic' 'Maureen Drake' 'Serena Williams' ... 'Mariya Vyshkina'
 'Anna Sharandachenko' 'Mariya Sharandachenko']


In [11]:
df_player.to_csv("../data/integrations/matches_female_players.csv", index=False)

## Start from here

In [12]:
male_stats = pd.read_csv("../data/integrations/matches_male_players.csv")
female_stats = pd.read_csv("../data/integrations/matches_female_players.csv")

Idee:

- Dividere le statistiche dei player in base al tipo di surface dove giocano, come fatto nel sito https://www.atptour.com/en/players/novak-djokovic/d643/player-stats
- Fare un'analisi separata maschi/femmine e poi confrontarla con le performance che ci sono nei tornei misti (se ne troviamo)



### Dataframe Player_game__statistics

- % ace per punto al servizio
- % breakpoint salvati

### Dataframe TOP Player

In [13]:
TOP = 50

In [14]:
#Input:
# - df, the dataframe you want to work on
# - attr, the attribute you want to sum row by row
# - group_by, the attribute used to group the dataframe before summing (Here we sum rows with the same "group_by" value)
# - top_list, this other dataframe is left joined with df to restrict its result only to the top_list
# - top_list_attr, it's needed in the join function to know how's called the attribute

def top_players_feature(df, attr, group_by, top_list, top_list_attr):
    #count 
    tmp = pd.DataFrame(df[df[attr]>=0].groupby(group_by, as_index=False)[attr].sum())
    #intersection with top_list
    tmp = top_list.join(tmp.set_index(group_by), on=top_list_attr, how='left')
    
    return tmp

#Function to generate the sum of a column, with a condition, grouped by the attribute given in input

def sum_by_surface(df, attr, group_by, top_list, top_list_attr):
    #count 
    tmp = pd.DataFrame(df[df[attr]>=0].groupby([group_by, 'surface'], as_index=False)[attr].sum())
    #intersection with top_list
    tmp = top_list.join(tmp.set_index(group_by), on=top_list_attr, how='left')
    
    return [tmp[tmp['surface'] == 'Clay'], tmp[tmp['surface'] == 'Grass'], tmp[tmp['surface'] == 'Hard'], tmp[tmp['surface'] == 'Carpet']]

In [15]:
# take the top players according to number of won tournament
df_top_male_players = pd.DataFrame(male_stats.sort_values(by='Won_Tournaments', ascending=False).head(TOP)['Name'])

#iterating for every player
#for name,index in df_top_male_players.iterrows():
#df_matches.groupby('winner_name').count()['w_ace'] 

df_top_male_player_matches_win = df_matches.join(df_top_male_players.set_index('Name'), on='winner_name', how='inner') #Ci sono partite in cui i winner sono loser e non vengono considerate
df_top_male_player_matches_los = df_matches.join(df_top_male_players.set_index('Name'), on='loser_name', how='inner') #Ci sono partite in cui i winner sono loser e non vengono considerate
df_top_male_player_matches  = pd.concat([df_top_male_player_matches_win, df_top_male_player_matches_los])
#df_top_male_players['w_ace'] = sum_by_condition(df_top_male_player_matches, 'w_ace', 'winner_name', df_top_male_players, 'Name')['w_ace']
# Ciclo for su lista di attributi che chiama sum_by_condition

attrs = ['w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon', 'w_SvGms', 'w_bpSaved', 'w_bpFaced']

for attr in attrs:
    #df_top_male_players[attr] = top_players_feature(df_top_male_player_matches, attr, 'winner_name', df_top_male_players, 'Name')[attr]
    result = sum_by_surface(df_top_male_player_matches, attr, 'winner_name', df_top_male_players, 'Name')
    df_top_male_players[attr + '_clay'] = result[0][attr]
    df_top_male_players[attr + '_clay'].fillna(0, inplace=True)
    df_top_male_players[attr + '_grass'] = result[1][attr]
    df_top_male_players[attr + '_grass'].fillna(0, inplace=True)
    df_top_male_players[attr + '_hard'] = result[2][attr]
    df_top_male_players[attr + '_hard'].fillna(0, inplace=True)
    df_top_male_players[attr + '_carpet'] = result[2][attr]
    df_top_male_players[attr + '_carpet'].fillna(0, inplace=True)

df_top_male_players

,Name,w_ace_clay,w_ace_grass,w_ace_hard,w_ace_carpet,w_df_clay,w_df_grass,w_df_hard,w_df_carpet,w_svpt_clay,...,w_SvGms_hard,w_SvGms_carpet,w_bpSaved_clay,w_bpSaved_grass,w_bpSaved_hard,w_bpSaved_carpet,w_bpFaced_clay,w_bpFaced_grass,w_bpFaced_hard,w_bpFaced_carpet
192,Novak Djokovic,456.0,347.0,1226.0,1226.0,244.0,147.0,569.0,569.0,9237.0,...,2583.0,2583.0,445.0,152.0,588.0,588.0,654.0,208.0,859.0,859.0
154,Rafael Nadal,337.0,114.0,801.0,801.0,269.0,32.0,336.0,336.0,10917.0,...,2139.0,2139.0,537.0,47.0,450.0,450.0,760.0,64.0,647.0,647.0
0,Alexander Zverev,712.0,323.0,1978.0,1978.0,389.0,88.0,781.0,781.0,8221.0,...,2535.0,2535.0,310.0,75.0,642.0,642.0,508.0,102.0,915.0,915.0
35,Roger Federer,114.0,551.0,1414.0,1414.0,27.0,64.0,278.0,278.0,1563.0,...,1943.0,1943.0,57.0,150.0,375.0,375.0,81.0,196.0,529.0,529.0
766,Daniil Medvedev,166.0,361.0,2248.0,2248.0,100.0,98.0,790.0,790.0,2496.0,...,2947.0,2947.0,118.0,84.0,710.0,710.0,180.0,123.0,1030.0,1030.0
679,Dominic Thiem,735.0,139.0,1201.0,1201.0,412.0,63.0,433.0,433.0,12133.0,...,2241.0,2241.0,517.0,47.0,542.0,542.0,765.0,60.0,805.0,805.0
188,Andy Murray,173.0,220.0,759.0,759.0,84.0,38.0,318.0,318.0,2609.0,...,1394.0,1394.0,126.0,88.0,343.0,343.0,181.0,128.0,522.0,522.0
1051,Andrey Rublev,287.0,119.0,1585.0,1585.0,135.0,32.0,560.0,560.0,5121.0,...,2411.0,2411.0,261.0,42.0,629.0,629.0,405.0,68.0,901.0,901.0
1366,Ugo Humbert,33.0,170.0,923.0,923.0,24.0,75.0,310.0,310.0,712.0,...,1340.0,1340.0,25.0,88.0,362.0,362.0,48.0,118.0,480.0,480.0
546,James Duckworth,74.0,136.0,1028.0,1028.0,27.0,26.0,250.0,250.0,713.0,...,1330.0,1330.0,49.0,63.0,314.0,314.0,62.0,76.0,425.0,425.0


In [16]:
df_top_male_players.to_csv("../data/integrations/male_winners_stats_top.csv", index=False)

In [17]:
male_stats

,Name,Sex,Hand,IOC,Won_Tournaments,Number_of_Matches,Win_Percentage,best_year_matches,worst_year_matches,best_year_wins,worst_year_wins
0,Alexander Zverev,M,R,GER,17.0,378.0,0.706349,2017.0,2020.0,2018.0,2020.0
1,Agustin Moreno,M,R,MEX,0.0,4.0,0.250000,2016.0,2016.0,2016.0,2016.0
2,Martin Damm,M,L,USA,0.0,7.0,0.285714,2021.0,2018.0,2021.0,2021.0
3,Gergely Kisgyorgy,M,R,HUN,0.0,2.0,0.500000,2016.0,2016.0,2016.0,2016.0
4,Frank Moser,M,R,GER,0.0,11.0,0.454545,2016.0,2017.0,2016.0,2016.0
...,...,...,...,...,...,...,...,...,...,...,...
3013,Lui Maxted,M,U,GBR,0.0,2.0,0.000000,2021.0,2021.0,NaN,NaN
3014,Lorenzo Angelini,M,U,ITA,0.0,1.0,0.000000,2021.0,2021.0,NaN,NaN
3015,Michele Vianello,M,R,ITA,0.0,2.0,0.000000,2021.0,2021.0,NaN,NaN
3016,William Jansen,M,L,GBR,0.0,1.0,0.000000,2021.0,2021.0,NaN,NaN


In [18]:
# DO the same for womens
# take the top players according to number of won tournament
df_top_female_players = pd.DataFrame(female_stats.sort_values(by='Won_Tournaments', ascending=False).head(TOP)['Name'])

#iterating for every player
#for name,index in df_top_female_players.iterrows():
#df_matches.groupby('winner_name').count()['w_ace'] 

df_top_female_player_matches_win = df_matches.join(df_top_female_players.set_index('Name'), on='winner_name', how='inner') #Ci sono partite in cui i winner sono loser e non vengono considerate
df_top_female_player_matches_los = df_matches.join(df_top_female_players.set_index('Name'), on='loser_name', how='inner') #Ci sono partite in cui i winner sono loser e non vengono considerate
df_top_female_player_matches  = pd.concat([df_top_female_player_matches_win, df_top_female_player_matches_los])
#df_top_female_players['w_ace'] = sum_by_condition(df_top_female_player_matches, 'w_ace', 'winner_name', df_top_female_players, 'Name')['w_ace']
# Ciclo for su lista di attributi che chiama sum_by_condition

attrs = ['w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon', 'w_SvGms', 'w_bpSaved', 'w_bpFaced']

#sum_by_condition(df_top_female_player_matches, 'w_svpt', 'winner_name', df_top_female_players, 'Name')

#sum_by_surface(df_top_female_player_matches, 'w_svpt', 'winner_name', df_top_female_players, 'Name', 'Clay')

for attr in attrs:
    #df_top_female_players[attr] = top_players_feature(df_top_female_player_matches, attr, 'winner_name', df_top_female_players, 'Name')[attr]
    result = sum_by_surface(df_top_female_player_matches, attr, 'winner_name', df_top_female_players, 'Name')
    df_top_female_players[attr + '_clay'] = result[0][attr]
    df_top_female_players[attr + '_clay'].fillna(0, inplace=True)
    df_top_female_players[attr + '_grass'] = result[1][attr]
    df_top_female_players[attr + '_grass'].fillna(0, inplace=True)
    df_top_female_players[attr + '_hard'] = result[2][attr]
    df_top_female_players[attr + '_hard'].fillna(0, inplace=True)
    df_top_female_players[attr + '_carpet'] = result[2][attr]
    df_top_female_players[attr + '_carpet'].fillna(0, inplace=True)

df_top_female_players

,Name,w_ace_clay,w_ace_grass,w_ace_hard,w_ace_carpet,w_df_clay,w_df_grass,w_df_hard,w_df_carpet,w_svpt_clay,...,w_SvGms_hard,w_SvGms_carpet,w_bpSaved_clay,w_bpSaved_grass,w_bpSaved_hard,w_bpSaved_carpet,w_bpFaced_clay,w_bpFaced_grass,w_bpFaced_hard,w_bpFaced_carpet
722,Fernanda Brito,2.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,166.0,...,0.0,0.0,5.0,0.0,0.0,0.0,13.0,0.0,0.0,0.0
118,Arantxa Rus,57.0,31.0,90.0,90.0,98.0,19.0,114.0,114.0,2254.0,...,408.0,408.0,128.0,25.0,166.0,166.0,216.0,35.0,261.0,261.0
434,Nuria Parrizas Diaz,2.0,6.0,1.0,1.0,23.0,8.0,3.0,3.0,1135.0,...,21.0,21.0,67.0,3.0,5.0,5.0,115.0,6.0,12.0,12.0
1494,Tamara Zidansek,69.0,1.0,43.0,43.0,152.0,6.0,69.0,69.0,4226.0,...,306.0,306.0,284.0,10.0,142.0,142.0,481.0,27.0,228.0,228.0
1578,Ylona Georgiana Ghioroaie,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
248,Ashleigh Barty,269.0,237.0,993.0,993.0,136.0,117.0,464.0,464.0,3280.0,...,1680.0,1680.0,163.0,121.0,555.0,555.0,244.0,175.0,859.0,859.0
272,Elina Svitolina,124.0,41.0,721.0,721.0,134.0,44.0,467.0,467.0,3770.0,...,1990.0,1990.0,227.0,64.0,701.0,701.0,375.0,99.0,1163.0,1163.0
1523,Aryna Sabalenka,128.0,95.0,795.0,795.0,143.0,91.0,667.0,667.0,2465.0,...,1598.0,1598.0,159.0,110.0,572.0,572.0,234.0,166.0,877.0,877.0
176,Karolina Pliskova,367.0,376.0,1239.0,1239.0,187.0,109.0,541.0,541.0,3804.0,...,1822.0,1822.0,191.0,113.0,651.0,651.0,299.0,163.0,972.0,972.0
142,Simona Halep,145.0,46.0,293.0,293.0,208.0,53.0,306.0,306.0,5456.0,...,1392.0,1392.0,303.0,89.0,446.0,446.0,514.0,143.0,770.0,770.0


In [19]:
df_top_female_players.to_csv("../data/integrations/female_winners_stats_top.csv", index=False)

### TODO: for losers?

### New Features analysis

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=de87b132-0371-4d99-94f9-c61923e2507d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>